# Auxiliary classifier using annotation

## 1. Data pre-processing: 
"train.txt" and "test.txt" are the same as "train.id" and "test.id"in ./xai_data_raw_apex_reg_cls

Generate: "train_data.json", "test_data.json", "train_label_id.json", "test_label_id.json"
  
format: dictionary  
data:  {file_name:annotation}  
label: {file_name:player_id}

## 1.1 Using peak value from kernel density estimation as label
Data source: "midi_label_map_apex_reg_cls.json"generated by running map_midi_to_label.py    

In [1]:
#Data processing: dict:key:file_name, value:player annotation
import json
# Opening JSON file
f = open('midi_label_map_apex_reg_cls.json')
# returns JSON object as  a dictionary
data = json.load(f)
  
# Iterating through the json
# list
print("length ", len(data))
print("length", len(data["Schubert_D960_mv2_8bars_Score_10"]))

train_dict ={}

with open('train.txt') as f:
    lines = f.readlines()
    print(len(lines))
    for i in lines:
        #print(i)
        i=i.strip('\n')
        train_dict[i]=data[i][:-1]
        
        #print(train_dict[i])


json.dump( train_dict, open( "train_data.json", 'w' ) )
#f2.close()

# Closing file
f.close()
print("FINISHED!:D")

length  846
length 26
719
FINISHED!:D


In [2]:
#dict:key:file_name, value:player annotation
import json
# Opening JSON file
f = open('midi_label_map_apex_reg_cls.json')
# returns JSON object as  a dictionary
data = json.load(f)
  
# Iterating through the json
# list
print("length ", len(data))
print("length", len(data["Schubert_D960_mv2_8bars_Score_10"]))

test_dict ={}

with open('test.txt') as f:
    lines = f.readlines()
    for i in lines:
        print(i)
        i=i.strip('\n')
        test_dict[i]=data[i][:-1]
        
        #print(test_dict[i])


json.dump(test_dict, open( "test_data.json", 'w' ) )


# Closing file
f.close()
print("FINISHED!:D")

length  846
length 26
Schubert_D960_mv3_8bars_9_18

Schubert_D960_mv3_8bars_1_19

Schubert_D960_mv3_8bars_1_11

Schubert_D960_mv2_8bars_2_01

Schubert_D960_mv2_8bars_10_04

Schubert_D960_mv2_8bars_10_07

Schubert_D960_mv2_8bars_9_12

Schubert_D960_mv3_8bars_5_17

Schubert_D960_mv3_8bars_5_34

Schubert_D960_mv3_8bars_8_29

Schubert_D960_mv3_8bars_5_35

Schubert_D960_mv3_8bars_1_09

Schubert_D960_mv3_8bars_Score_09

Schubert_D960_mv3_8bars_Score_12

Schubert_D960_mv3_8bars_7_11

Schubert_D960_mv3_8bars_7_37

Schubert_D960_mv3_8bars_8_28

Schubert_D960_mv3_8bars_7_29

Schubert_D960_mv3_8bars_10_13

Schubert_D960_mv3_8bars_5_32

Schubert_D960_mv3_8bars_9_08

Schubert_D960_mv2_8bars_5_04

Schubert_D960_mv2_8bars_1_17

Schubert_D960_mv2_8bars_2_11

Schubert_D960_mv2_8bars_0_03

Schubert_D960_mv3_8bars_7_17

Schubert_D960_mv3_8bars_12_32

Schubert_D960_mv2_8bars_10_10

Schubert_D960_mv3_16bars_1_12

Schubert_D960_mv3_16bars_5_19

Schubert_D960_mv2_8bars_12_02

Schubert_D960_mv2_8bars_0_11

Sc

In [3]:
#dict:key:file_name, value:one-hot encoding player id
import json
# Opening JSON file
f = open('train_data.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
print("length ", len(data))
for file_name in data:
    player_id = (file_name.split('/')[-1].split('.')[0].split("_")[-2])
    #print("file_name:", file_name)
    #print("id:", player_id)
    if player_id == "Score":
        #print("SCORE")
        player_id = 11
    else:
        player_id = int(player_id)
    #data[file_name]= [0] * 13
    #data[file_name][player_id] = 1
    data[file_name]= player_id
    #print(data[file_name])


print(len(data))
data_temp = json.dumps(data)
f2 = open('train_label_id.json', 'w')
f2.write(data_temp)
f2.close()

# Closing file
f.close()
print("FINISHED!:D")

length  719
719
FINISHED!:D


In [4]:
#dict:key:file_name, value:one-hot encoding player id
import json
# Opening JSON file
f = open('test_data.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
print("length ", len(data))
for file_name in data:
    player_id = (file_name.split('/')[-1].split('.')[0].split("_")[-2])
    #print("file_name:", file_name)
    #print("id:", player_id)
    if player_id == "Score":
        #print("SCORE")
        player_id = 11
    else:
        player_id = int(player_id)
    #data[file_name]= [0] * 13
    #data[file_name][player_id] = 1
    data[file_name]= player_id
    #print(data[file_name])


print(len(data))
data_temp = json.dumps(data)
f2 = open('test_label_id.json', 'w')
f2.write(data_temp)
f2.close()

# Closing file
f.close()
print("FINISHED!:D")

length  80
80
FINISHED!:D


## 1.2 Using all annotation data as label

In [5]:
import csv
import numpy as np
import json

f_train = open('train.txt')
f_train_data = f_train.readlines()

f_test = open('test.txt')
f_test_data = f_test.readlines()

file = open('./total.csv', encoding="utf-8")
#csvreader = csv.reader(file)
line_index = 0

train_data_list = []
test_data_list = []

train_label_list = []
test_label_list = []
# level = professional

with file as myFile:  
    lines=csv.reader(myFile)  
    for line in lines:
        line_index +=1
        if line_index == 1:
            print(line)
            line = line[3:31]
            print(line)
            continue
        #print(line)
        file_name = line[2][:-4]
        player_id = (file_name.split('/')[-1].split('.')[0].split("_")[-2])
        user_id = int(line[0])
        #print(user_id)
        #print(line[1])
        #print("*",line_index)
        #print("file_name:", file_name)
        #print("id:", player_id)
        if player_id == "Score":
            #print("SCORE")
            player_id = 11
        else:
            player_id = int(player_id)
        #line = line[3:-2]
        line = [line[3]]+line[6:31]
        #line = line[3:31]
        for i in range(0, len(line)):
            if line[i] == '':
                line[i] = 0
            line[i] = float(line[i])
            
        # for training data
  
        if file_name+'\n' in f_train_data:
            #print("Train data!")
            train_data_list.append(line)
            train_label_list.append(player_id)
            #print("*",user_id)
        elif  file_name+'\n' in f_test_data:
            #print("Test data!")
            test_data_list.append(line)
            test_label_list.append(player_id)
            #print("*",user_id)
        #else:
            #print(file_name)
            #assert False

print(len(train_data_list))
print(len(train_label_list))
print(len(test_data_list))
print(len(test_label_list))



train_data_list_temp = json.dumps(train_data_list)
train_label_list_temp = json.dumps(train_label_list)
test_data_list_temp = json.dumps(test_data_list)
test_label_list_temp = json.dumps(test_label_list)
f_train_data_list = open('cls_train_data_list_all.json', 'w')
f_train_data_list.write(train_data_list_temp)
f_train_data_list.close()

f_train_label_list = open('cls_train_label_list_all.json', 'w')
f_train_label_list.write(train_label_list_temp)
f_train_label_list.close()

f_test_data_list= open('cls_test_data_list_all.json', 'w')
f_test_data_list.write(test_data_list_temp)
f_test_data_list.close()

f_test_label_list = open('cls_test_label_list_all.json', 'w')
f_test_label_list.write(test_label_list_temp)
f_test_label_list.close()

print("Finished!:D")

['user', 'dataID', 'filename', 'Question_1_1_1', 'Question_1_2_1', 'Question_1_2_2', 'Question_1_3_1', 'Question_2_1_1', 'Question_2_2_1', 'Question_3_1_1', 'Question_3_2_1', 'Question_3_3_1', 'Question_4_1_1', 'Question_4_2_1', 'Question_4_3_1', 'Question_5_1_1', 'Question_5_2_1', 'Question_5_3_1', 'Question_5_4_1', 'Question_5_5_1', 'Question_6_1_1', 'Question_6_2_1', 'Question_6_3_1', 'Question_6_4_1', 'Question_6_5_1', 'Question_7_1_1', 'Question_7_2_1', 'Question_7_3_1', 'Question_8_1_1', 'Question_8_2_1', 'Question_9_1_1', 'work_length', 'message']
['Question_1_1_1', 'Question_1_2_1', 'Question_1_2_2', 'Question_1_3_1', 'Question_2_1_1', 'Question_2_2_1', 'Question_3_1_1', 'Question_3_2_1', 'Question_3_3_1', 'Question_4_1_1', 'Question_4_2_1', 'Question_4_3_1', 'Question_5_1_1', 'Question_5_2_1', 'Question_5_3_1', 'Question_5_4_1', 'Question_5_5_1', 'Question_6_1_1', 'Question_6_2_1', 'Question_6_3_1', 'Question_6_4_1', 'Question_6_5_1', 'Question_7_1_1', 'Question_7_2_1', 'Quest

## 1.3 Using annotations form professional annotator

In [6]:
import csv
import numpy as np
import json

f_train = open('train.txt')
f_train_data = f_train.readlines()

f_test = open('test.txt')
f_test_data = f_test.readlines()

file = open('./total.csv', encoding="utf-8")
#csvreader = csv.reader(file)
line_index = 0

train_data_list = []
test_data_list = []

train_label_list = []
test_label_list = []
# level = professional
annotator_id = [3,7,8,25,26,30,32,33,35,38,40,41,42,43,45,48,49,74,79,81,82,85,86,88,92,95,96,98,99,108,109,110,112,114,115,117,121]


with file as myFile:  
    lines=csv.reader(myFile)  
    for line in lines:
        line_index +=1
        if line_index == 1:
            print(line)
            line = line[3:31]
            print(line)
            continue
        #print(line)
        file_name = line[2][:-4]
        player_id = (file_name.split('/')[-1].split('.')[0].split("_")[-2])
        user_id = int(line[0])
        #print(user_id)
        #print(line[1])
        #print("*",line_index)
        #print("file_name:", file_name)
        #print("id:", player_id)
        if player_id == "Score":
            #print("SCORE")
            player_id = 11
        else:
            player_id = int(player_id)
        #line = line[3:-2]
        line = [line[3]]+line[6:31]
        #line = line[3:31]
        for i in range(0, len(line)):
            if line[i] == '':
                line[i] = 0
            line[i] = float(line[i])
            
        # for training data
        if user_id in annotator_id:
            if file_name+'\n' in f_train_data:
                #print("Train data!")
                train_data_list.append(line)
                train_label_list.append(player_id)
                #print("*",user_id)
            elif  file_name+'\n' in f_test_data:
                #print("Test data!")
                test_data_list.append(line)
                test_label_list.append(player_id)
                #print("*",user_id)
            #else:
                #print(file_name)
                #assert False

print(len(train_data_list))
print(len(train_label_list))
print(len(test_data_list))
print(len(test_label_list))



train_data_list_temp = json.dumps(train_data_list)
train_label_list_temp = json.dumps(train_label_list)
test_data_list_temp = json.dumps(test_data_list)
test_label_list_temp = json.dumps(test_label_list)
f_train_data_list = open('cls_train_data_list_prof.json', 'w')
f_train_data_list.write(train_data_list_temp)
f_train_data_list.close()

f_train_label_list = open('cls_train_label_list_prof.json', 'w')
f_train_label_list.write(train_label_list_temp)
f_train_label_list.close()

f_test_data_list= open('cls_test_data_list_prof.json', 'w')
f_test_data_list.write(test_data_list_temp)
f_test_data_list.close()

f_test_label_list = open('cls_test_label_list_prof.json', 'w')
f_test_label_list.write(test_label_list_temp)
f_test_label_list.close()

print("Finished!:D")

['user', 'dataID', 'filename', 'Question_1_1_1', 'Question_1_2_1', 'Question_1_2_2', 'Question_1_3_1', 'Question_2_1_1', 'Question_2_2_1', 'Question_3_1_1', 'Question_3_2_1', 'Question_3_3_1', 'Question_4_1_1', 'Question_4_2_1', 'Question_4_3_1', 'Question_5_1_1', 'Question_5_2_1', 'Question_5_3_1', 'Question_5_4_1', 'Question_5_5_1', 'Question_6_1_1', 'Question_6_2_1', 'Question_6_3_1', 'Question_6_4_1', 'Question_6_5_1', 'Question_7_1_1', 'Question_7_2_1', 'Question_7_3_1', 'Question_8_1_1', 'Question_8_2_1', 'Question_9_1_1', 'work_length', 'message']
['Question_1_1_1', 'Question_1_2_1', 'Question_1_2_2', 'Question_1_3_1', 'Question_2_1_1', 'Question_2_2_1', 'Question_3_1_1', 'Question_3_2_1', 'Question_3_3_1', 'Question_4_1_1', 'Question_4_2_1', 'Question_4_3_1', 'Question_5_1_1', 'Question_5_2_1', 'Question_5_3_1', 'Question_5_4_1', 'Question_5_5_1', 'Question_6_1_1', 'Question_6_2_1', 'Question_6_3_1', 'Question_6_4_1', 'Question_6_5_1', 'Question_7_1_1', 'Question_7_2_1', 'Quest

## 1.4 Using mean value as label

In [7]:
import csv
from collections import OrderedDict, defaultdict
import numpy as np
import json
from scipy.stats import gaussian_kde
from tqdm import tqdm

LABEL_LIST = ["Stable beat", "Mechanical Tempo", "Intensional", "Regular beat change", "Long", "Cushioned", "Saturated (wet)", "Clean", "Subtle change", "Even", "Rich", "Bright", 
"Pure", "Soft", "Sophisticated(mellow)", "balanced", "Large range of dynamic", "Fast paced", "Flowing", "Swing(Flexible)", "Flat", "Harmonious", "Optimistic(pleasant)", "HIgh Energy", 
"Dominant(forceful)", "Imaginative", "Ethereal", "Convincing"]
LABEL_MAP = {i: label for i, label in enumerate(LABEL_LIST)}
PIANIST_MAP = OrderedDict()

file = open('total.csv', encoding="utf-8")
#file = open('total.csv', encoding="utf-8")

def estimate_maxima(data):
    if len(set(data))<=1: # all datas are equal
        return data[0]
    kde = gaussian_kde(data)
    no_samples = 50
    samples = np.linspace(min(data), max(data), no_samples)
    probs = kde.evaluate(samples)
    #maxima_index = probs.argmax()
    # in case if more than 1 argmaxs
    winner = np.argwhere(probs == np.amax(probs))
    maxima = np.average(samples[winner.flatten()])
    return maxima

# model can't predict these labels well
# regular beat change, → remove
# subtle change → remove
# sophisticated,  (not sure) balanced, ~~harmonious~~ → select one
# Dominant → related with high energy. delete
# ethereal, imaginative, convincing → select one

def midi_label_map_apex_remove_bad():
    LABEL_TO_REMOVE = ["Regular beat change", "Subtle change", "Sophisticated(mellow)", "balanced", "Harmonious", "Dominant(forceful)", "Imaginative"]
    print(len(LABEL_TO_REMOVE))
    filtered_loc = [LABEL_LIST.index(name) for name in LABEL_TO_REMOVE]
    filtered_loc = [loc for loc in list(range(0,28)) if loc not in filtered_loc]
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)

    rows = []
    for row in csvreader:
        rows.append(row)

    # sort by each segments
    music_label_map = defaultdict(list)
    for row in rows:
        user = row[0]
        file_name = row[2].split(".")[0]
        label_row = row[3:-2]
        # label_row = row[6:-2] 
        label_row = [label_row[loc] for loc in filtered_loc]
        label_row = label_row[3:] # skip 1-1 ~ 1-3
        print(len(label_row))
        for idx, elem in enumerate(label_row):
            if elem == "":
                label_row[idx] = 0.0
            else:
                label_row[idx] = float(elem)
        # skip 0
        if 0.0 in label_row:
            continue
        else:
            music_label_map[file_name].append(label_row)

    music_label_map_apex = dict()

    # kernel density estimation
    for key, annot_list in tqdm(music_label_map.items()):
        annot_list = np.array(annot_list).transpose()
        maxima = np.array([estimate_maxima(row)/7 for row in annot_list])
        maxima = maxima.transpose().tolist()
        music_label_map_apex[key] = maxima

    # add pianist info
    for key, annot_list in tqdm(music_label_map_apex.items()):
        if key.split("_")[-2] not in PIANIST_MAP:
            PIANIST_MAP[key.split("_")[-2]] = len(PIANIST_MAP)
    print(PIANIST_MAP)
    
    for key, annot_list in tqdm(music_label_map_apex.items()):
        music_label_map_apex[key].append(PIANIST_MAP[key.split("_")[-2]])

    json.dump(music_label_map_apex, open("midi_label_map_apex_rm_bad.json", 'w'))

def midi_label_map_apex():

    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)

    rows = []
    for row in csvreader:
        rows.append(row)

    # sort by each segments
    music_label_map = defaultdict(list)
    for row in rows:
        user = row[0]
        file_name = row[2].split(".")[0]
        #label_row = row[3:-2]
        label_row = [row[3]] + row[7:-2] # skip 1-2 ~ 1-3
        for idx, elem in enumerate(label_row):
            if elem == "":
                label_row[idx] = 0.0
            else:
                label_row[idx] = float(elem)
        # skip 0
        if 0.0 in label_row:
            continue
        else:
            music_label_map[file_name].append(label_row)

    music_label_map_apex = dict()

    # kernel density estimation
    for key, annot_list in tqdm(music_label_map.items()):
        annot_list = np.array(annot_list).transpose()
        maxima = np.mean(annot_list, axis = 1)
        maxima = maxima.tolist()
        music_label_map_apex[key] = maxima



    json.dump(music_label_map_apex, open("midi_label_map_apex_reg_cls_avg.json", 'w'))
    
if __name__ == "__main__":
    #midi_label_map_apex_filtered() 
    #midi_label_map_apex_except_filtered() 
    midi_label_map_apex() 
    #midi_label_map_apex_remove_bad() 
    #midi_label_map_stdev() 
    #midi_label_map_apex_base_split()

100%|█████████████████████████████████████████████████████████████████████████████| 846/846 [00:00<00:00, 56404.09it/s]


In [8]:
#Data processing: dict:key:file_name, value:player annotation
import json
# Opening JSON file
f = open('midi_label_map_apex_reg_cls_avg.json')
# returns JSON object as  a dictionary
data = json.load(f)
  
# Iterating through the json
# list
print("length ", len(data))
print("length", len(data["Schubert_D960_mv2_8bars_Score_10"]))

train_dict ={}

with open('train.txt') as f:
    lines = f.readlines()
    print(len(lines))
    for i in lines:
        #print(i)
        i=i.strip('\n')
        train_dict[i]=data[i]
        
        #print(train_dict[i])


json.dump( train_dict, open( "train_data_avg.json", 'w' ) )
#f2.close()

# Closing file
f.close()
print("FINISHED!:D")

length  846
length 25
719
FINISHED!:D


In [9]:
#Data processing: dict:key:file_name, value:player annotation
import json
# Opening JSON file
f = open('midi_label_map_apex_reg_cls_avg.json')
# returns JSON object as  a dictionary
data = json.load(f)
  
# Iterating through the json
# list
print("length ", len(data))
print("length", len(data["Schubert_D960_mv2_8bars_Score_10"]))

train_dict ={}

with open('test.txt') as f:
    lines = f.readlines()
    print(len(lines))
    for i in lines:
        #print(i)
        i=i.strip('\n')
        train_dict[i]=data[i]
        
        #print(train_dict[i])


json.dump( train_dict, open( "test_data_avg.json", 'w' ) )
#f2.close()

# Closing file
f.close()
print("FINISHED!:D")

length  846
length 25
80
FINISHED!:D


## 2. Auxiliary_classifier:

## 2.1 Using peak of kernel density estimation


In [10]:
#using the  kernel density estimation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import json

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)
f_train_data = open('train_data.json')
f_test_data = open('test_data.json')
f_train_label = open('train_label_id.json')
f_test_label = open('test_label_id.json')

train_data = json.load(f_train_data)
test_data = json.load(f_test_data)
train_label = json.load(f_train_label)
test_label = json.load(f_test_label)

train_data_value = list(train_data.values())
test_data_value = list(test_data.values())
train_label_value = list(train_label.values())
test_label_value = list(test_label.values())
#print(test_label_value)


svc = svm.SVC()
svc.fit(train_data_value,train_label_value)
prediction = svc.predict(test_data_value)

#print("SVC", prediction)

result= (test_label_value == prediction)
#print(result)
#print(result.sum()/len(result))
print("SVM",svc.score(test_data_value,test_label_value))


clf_1 = RandomForestClassifier(n_estimators=10)
clf_1 = clf_1.fit(train_data_value, train_label_value)
scores = clf_1.score(test_data_value, test_label_value)
print("RandomForest",scores)


clf_2 = AdaBoostClassifier(n_estimators=10)
clf_2 = clf_2.fit(train_data_value, train_label_value)
scores = clf_2.score(test_data_value, test_label_value)
print("Adaboost",scores)

clf_3 = DecisionTreeClassifier()
clf_3 = clf_3.fit(train_data_value,train_label_value)
scores = clf_3.score(test_data_value, test_label_value)
print("DecisionTree",scores)



clf_4 = SGDClassifier(loss="hinge", penalty="l2", max_iter=50000)
clf_4 = clf_4.fit(train_data_value,train_label_value)
scores = clf_4.score(test_data_value, test_label_value)
print("SGDClassifier",scores)


clf_5 = MLPClassifier(random_state=1, max_iter=20000)
clf_5 = clf_5.fit(train_data_value,train_label_value)
#prob = clf_5.predict_log_proba(test_data_value)
#print(softmax(prob))
#prob_max = softmax(prob).argmax(axis=-1)
#print(prob_max)
scores = clf_5.score(test_data_value, test_label_value)
print("MLPClassifier",scores)


clf_6 = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf_6 = clf_6.fit(train_data_value,train_label_value)
scores = clf_6.score(test_data_value, test_label_value)
print("ExtraTreesClassifier",scores)

clf_7 = RandomForestClassifier(max_depth=5, random_state=0)
clf_7 = clf_7.fit(train_data_value,train_label_value)
scores = clf_7.score(test_data_value, test_label_value)
print("RandomForestClassifier",scores)

f_train_data.close()
f_test_data.close()
f_train_label.close()
f_test_label.close()
#print(train_label_value)


SVM 0.2
RandomForest 0.175
Adaboost 0.0875
DecisionTree 0.15
SGDClassifier 0.075
MLPClassifier 0.2375
ExtraTreesClassifier 0.1875
RandomForestClassifier 0.125


## 2.2 Using annotations form professional annotator

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import json


f_train_data = open('cls_train_data_list.json')
f_test_data = open('cls_test_data_list.json')
f_train_label = open('cls_train_label_list.json')
f_test_label = open('cls_test_label_list.json')

train_data_value = json.load(f_train_data)
test_data_value = json.load(f_test_data)
train_label_value = json.load(f_train_label)
test_label_value = json.load(f_test_label)

#print(test_label_value)

print(len(train_data_value))
print(len(test_data_value))
print(len(train_label_value))
print(len(test_label_value))

svc = svm.SVC()
svc.fit(train_data_value,train_label_value)
prediction = svc.predict(test_data_value)

#print("SVC", prediction)

result= (test_label_value == prediction)
#print(result)
#print(result.sum()/len(result))
print("SVM",svc.score(test_data_value,test_label_value))


clf_1 = RandomForestClassifier(n_estimators=10)
clf_1 = clf_1.fit(train_data_value, train_label_value)
scores = clf_1.score(test_data_value, test_label_value)
print("RandomForest",scores)


clf_2 = AdaBoostClassifier(n_estimators=10)
clf_2 = clf_2.fit(train_data_value, train_label_value)
scores = clf_2.score(test_data_value, test_label_value)
print("Adaboost",scores)

clf_3 = DecisionTreeClassifier()
clf_3 = clf_3.fit(train_data_value,train_label_value)
scores = clf_3.score(test_data_value, test_label_value)
print("DecisionTree",scores)



clf_4 = SGDClassifier(loss="hinge", penalty="l2", max_iter=10000)
clf_4 = clf_4.fit(train_data_value,train_label_value)
scores = clf_4.score(test_data_value, test_label_value)
print("SGDClassifier",scores)

clf_5 = MLPClassifier(random_state=1, max_iter=20000)
clf_5 = clf_5.fit(train_data_value,train_label_value)
scores = clf_5.score(test_data_value, test_label_value)
print("MLPClassifier",scores)


clf_6 = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf_6 = clf_6.fit(train_data_value,train_label_value)
scores = clf_6.score(test_data_value, test_label_value)
print("ExtraTreesClassifier",scores)

clf_7 = RandomForestClassifier(max_depth=5, random_state=0)
clf_7 = clf_7.fit(train_data_value,train_label_value)
scores = clf_7.score(test_data_value, test_label_value)
print("RandomForestClassifier",scores)


f_train_data.close()
f_test_data.close()
f_train_label.close()
f_test_label.close()

1901
217
1901
217
SVM 0.15668202764976957
RandomForest 0.1336405529953917
Adaboost 0.14746543778801843
DecisionTree 0.1152073732718894
SGDClassifier 0.055299539170506916
MLPClassifier 0.15668202764976957
ExtraTreesClassifier 0.16589861751152074
RandomForestClassifier 0.14746543778801843


## 2.3 Using all annotation data as label

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import json


f_train_data = open('cls_train_data_list_all.json')
f_test_data = open('cls_test_data_list_all.json')
f_train_label = open('cls_train_label_list_all.json')
f_test_label = open('cls_test_label_list_all.json')

train_data_value = json.load(f_train_data)
test_data_value = json.load(f_test_data)
train_label_value = json.load(f_train_label)
test_label_value = json.load(f_test_label)



print(len(train_data_value))
print(len(test_data_value))
print(len(train_label_value))
print(len(test_label_value))

svc = svm.SVC()
svc.fit(train_data_value,train_label_value)
prediction = svc.predict(test_data_value)

#print("SVC", prediction)

result= (test_label_value == prediction)
#print(result)
#print(result.sum()/len(result))
print("SVM",svc.score(test_data_value,test_label_value))


clf_1 = RandomForestClassifier(n_estimators=10)
clf_1 = clf_1.fit(train_data_value, train_label_value)
scores = clf_1.score(test_data_value, test_label_value)
print("RandomForest",scores)


clf_2 = AdaBoostClassifier(n_estimators=10)
clf_2 = clf_2.fit(train_data_value, train_label_value)
scores = clf_2.score(test_data_value, test_label_value)
print("Adaboost",scores)

clf_3 = DecisionTreeClassifier()
clf_3 = clf_3.fit(train_data_value,train_label_value)
scores = clf_3.score(test_data_value, test_label_value)
print("DecisionTree",scores)



clf_4 = SGDClassifier(loss="hinge", penalty="l2", max_iter=10000)
clf_4 = clf_4.fit(train_data_value,train_label_value)
scores = clf_4.score(test_data_value, test_label_value)
print("SGDClassifier",scores)

clf_5 = MLPClassifier(random_state=1, max_iter=20000,hidden_layer_sizes=(100,))
clf_5 = clf_5.fit(train_data_value,train_label_value)
scores = clf_5.score(test_data_value, test_label_value)
print("MLPClassifier",scores)


clf_6 = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf_6 = clf_6.fit(train_data_value,train_label_value)
scores = clf_6.score(test_data_value, test_label_value)
print("ExtraTreesClassifier",scores)

clf_7 = RandomForestClassifier(max_depth=5, random_state=0)
clf_7 = clf_7.fit(train_data_value,train_label_value)
scores = clf_7.score(test_data_value, test_label_value)
print("RandomForestClassifier",scores)


f_train_data.close()
f_test_data.close()
f_train_label.close()
f_test_label.close()

6190
701
6190
701
SVM 0.14550641940085593
RandomForest 0.14550641940085593
Adaboost 0.12553495007132667
DecisionTree 0.1369472182596291
SGDClassifier 0.1326676176890157
MLPClassifier 0.12696148359486448
ExtraTreesClassifier 0.16833095577746077
RandomForestClassifier 0.12553495007132667


## 2.4 Using mean value as label

In [14]:
#using the  kernel density estimation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import json

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)
f_train_data = open('train_data_avg.json')
f_test_data = open('test_data_avg.json')
f_train_label = open('train_label_id.json')
f_test_label = open('test_label_id.json')

train_data = json.load(f_train_data)
test_data = json.load(f_test_data)
train_label = json.load(f_train_label)
test_label = json.load(f_test_label)

train_data_value = list(train_data.values())
test_data_value = list(test_data.values())
train_label_value = list(train_label.values())
test_label_value = list(test_label.values())
#print(test_label_value)


svc = svm.SVC()
svc.fit(train_data_value,train_label_value)
prediction = svc.predict(test_data_value)

#print("SVC", prediction)

result= (test_label_value == prediction)
#print(result)
#print(result.sum()/len(result))
print("SVM",svc.score(test_data_value,test_label_value))


clf_1 = RandomForestClassifier(n_estimators=10)
clf_1 = clf_1.fit(train_data_value, train_label_value)
scores = clf_1.score(test_data_value, test_label_value)
print("RandomForest",scores)


clf_2 = AdaBoostClassifier(n_estimators=10)
clf_2 = clf_2.fit(train_data_value, train_label_value)
scores = clf_2.score(test_data_value, test_label_value)
print("Adaboost",scores)

clf_3 = DecisionTreeClassifier()
clf_3 = clf_3.fit(train_data_value,train_label_value)
scores = clf_3.score(test_data_value, test_label_value)
print("DecisionTree",scores)



clf_4 = SGDClassifier(loss="hinge", penalty="l2", max_iter=50000)
clf_4 = clf_4.fit(train_data_value,train_label_value)
scores = clf_4.score(test_data_value, test_label_value)
print("SGDClassifier",scores)


clf_5 = MLPClassifier(random_state=1, max_iter=20000)
clf_5 = clf_5.fit(train_data_value,train_label_value)
#prob = clf_5.predict_log_proba(test_data_value)
#print(softmax(prob))
#prob_max = softmax(prob).argmax(axis=-1)
#print(prob_max)
scores = clf_5.score(test_data_value, test_label_value)
print("MLPClassifier",scores)


clf_6 = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf_6 = clf_6.fit(train_data_value,train_label_value)
scores = clf_6.score(test_data_value, test_label_value)
print("ExtraTreesClassifier",scores)

clf_7 = RandomForestClassifier(max_depth=5, random_state=0)
clf_7 = clf_7.fit(train_data_value,train_label_value)
scores = clf_7.score(test_data_value, test_label_value)
print("RandomForestClassifier",scores)

f_train_data.close()
f_test_data.close()
f_train_label.close()
f_test_label.close()
#print(train_label_value)


SVM 0.1875
RandomForest 0.1375
Adaboost 0.175
DecisionTree 0.2125
SGDClassifier 0.1125
MLPClassifier 0.2
ExtraTreesClassifier 0.15
RandomForestClassifier 0.1875


## 3. Ensemble learning with Weighted Average Method:

The predicted probabilities of MusicBert and MLP are combined together as the final prediction score.

In [ ]:
##MLP
#using the  kernel density estimation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
import json
import numpy


f_train_data = open('train_data.json')
f_test_data = open('test_data.json')
f_train_label = open('train_label_id.json')
f_test_label = open('test_label_id.json')

train_data = json.load(f_train_data)
test_data = json.load(f_test_data)
train_label = json.load(f_train_label)
test_label = json.load(f_test_label)

train_data_value = list(train_data.values())
test_data_value = list(test_data.values())
train_label_value = list(train_label.values())
test_label_value = list(test_label.values())
#print(test_label_value)


clf_5 = MLPClassifier(random_state=1, max_iter=20000, hidden_layer_sizes =(100,),learning_rate_init=0.1)
#,hidden_layer_sizes=(200,), activation='relu',learning_rate='constant', learning_rate_init=0.001,solver = 'adam'
clf_5 = clf_5.fit(train_data_value,train_label_value)
prob = clf_5.predict_proba(test_data_value)
print(prob)
prob_max = prob.argmax(axis=-1)
print(prob_max)
scores = clf_5.score(test_data_value, test_label_value)
print("MLPClassifier",scores)

numpy.savetxt("MLPClassifier_test_prob.txt", prob)

content = numpy.loadtxt('MLPClassifier_test_prob.txt')
#print("\nContent in file1.txt:\n", content)


In [ ]:
import numpy as np
import torch.nn.functional as F
import torch

MLP_pred = np.loadtxt('MLPClassifier_test_prob.txt')
MusicBert_pred = np.loadtxt('MusicBert_pred.txt')
label = np.loadtxt('MusicBert_label.txt')
#print(MLP_pred)
'''
T = 2
MLP_pred = np.log(MLP_pred)
MLP_pred= torch.from_numpy(MLP_pred)
MLP_pred_softmax = F.softmax(MLP_pred/T,dim=-1)
MLP_pred = MLP_pred_softmax.numpy()
#print(MLP_pred)
T = 0.5
MusicBert_pred = np.log(MusicBert_pred)
MusicBert_pred= torch.from_numpy(MusicBert_pred)
MusicBert_pred = F.softmax(MusicBert_pred/T,dim=-1)
MusicBert_pred = MusicBert_pred.numpy()
'''

MLP_pred_max = MLP_pred.argmax(axis=-1)
MusicBert_pred_max = MusicBert_pred.argmax(axis=-1)
label_index = label.argmax(axis=-1)

result_MLP= (MLP_pred_max == label_index)
result_MusicBert= (MusicBert_pred_max == label_index)
#print(result)
acc_MLP = result_MLP.sum()/len(result_MLP)
acc_Musicbert = result_MusicBert.sum()/len(result_MLP)
print("MLP acc",acc_MLP)
print("MusicBert acc",acc_Musicbert)

'''
a = acc_MLP/(acc_MLP+acc_Musicbert)
print("ensemble weight calculated by acc", a)

combine = a* MLP_pred+(1-a)*MusicBert_pred
combine_max = combine.argmax(axis=-1)
result_combine= (combine_max == label_index)
acc = result_combine.sum()/len(result_combine)
#print("weight",a)
print("Ensemble acc",acc)
'''
alpha = np.linspace(0.,1,200000)
for a in alpha:
    combine = a* MLP_pred+(1-a)*MusicBert_pred
    combine_max = combine.argmax(axis=-1)
    result_combine= (combine_max == label_index)
    acc = result_combine.sum()/len(result_combine) 
    if acc>0.8375:
        print(acc)
print("FINISHED!:D")